In [13]:
import grpc
import tensorflow as tf
import numpy as np

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

Мы импортировали:
- `grpc` для связи с TF Serving
- `tensorflow` для определений protobuf
- пару функций из TF Serving

In [14]:
# Определяем подключение к сервису:

host = 'localhost:8502'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

Предварительная обработка изображений:

In [15]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [16]:
# Преобразовываем в массив Numpy
url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)
print("Loaded data:", X)

Loaded data: [[[[-0.11372548 -0.15294117 -0.19999999]
   [-0.11372548 -0.15294117 -0.19999999]
   [-0.10588235 -0.14509803 -0.19215685]
   ...
   [-0.01960784 -0.01960784 -0.08235294]
   [-0.04313725 -0.04313725 -0.10588235]
   [-0.11372548 -0.11372548 -0.17647058]]

  [[-0.09019607 -0.12941176 -0.17647058]
   [-0.09019607 -0.12941176 -0.17647058]
   [-0.08235294 -0.12156862 -0.16862744]
   ...
   [-0.01960784 -0.01960784 -0.08235294]
   [-0.04313725 -0.04313725 -0.10588235]
   [-0.10588235 -0.10588235 -0.16862744]]

  [[-0.09803921 -0.1372549  -0.18431371]
   [-0.09803921 -0.1372549  -0.18431371]
   [-0.09019607 -0.12941176 -0.17647058]
   ...
   [-0.01960784 -0.01960784 -0.08235294]
   [-0.03529412 -0.03529412 -0.09803921]
   [-0.09019607 -0.09019607 -0.15294117]]

  ...

  [[-0.67058825 -0.7019608  -0.7254902 ]
   [-0.6862745  -0.7176471  -0.7411765 ]
   [-0.70980394 -0.7411765  -0.7647059 ]
   ...
   [-0.62352943 -0.84313726 -0.9529412 ]
   [-0.6313726  -0.8509804  -0.9607843 ]
   

In [17]:
print("Type of X:", type(X))
print("Shape of X:", X.shape)
print("Dtype of X:", X.dtype)

Type of X: <class 'numpy.ndarray'>
Shape of X: (1, 299, 299, 3)
Dtype of X: float32


In [ ]:
# # Для gRPC нам необходимо преобразовать массив Numpy в protobuf c помощью специальной функции:
# 
# def np_to_protobuf(data):
#     return tf.make_tensor_proto(data, shape=data.shape)

In [ ]:
# pb_request = predict_pb2.PredictRequest()
# 
# pb_request.model_spec.name = 'clothing-model'
# pb_request.model_spec.signature_name = 'serving_default'
# pb_request.inputs['inputs'].CopyFrom(np_to_protobuf(X)) 

In [ ]:
# pb_result = stub.Predict(pb_request, timeout=20.0)

In [ ]:
# # используем функцию для подготовки запроса gRPC:
# pb_request = predict_pb2.PredictRequest() # Если есть один элемент, то использовать один элемент
# 
# # Установка имени модели и сигнатуры
# pb_request.model_spec.name = 'clothing-model' # Устанавливает имя модели в clohing-model
# pb_request.model_spec.signature_name = 'serving_default' # Задает имя сигнатуры
# 
# # Подготовка входных данных
# X = np.random.random((1, 299, 299, 3)).astype(np.float32)  # Пример входного массива c преобразованием в float32
# pb_request.inputs['inputs'].CopyFrom(np_to_protobuf(X)) # преобразует массив в protobuf
# 
# # Создание канала и stub
# channel = grpc.insecure_channel('localhost:8500')
# stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# 
# # Выполнение запроса
# try:
#     pb_result = stub.Predict(pb_request, timeout=20.0)
#     print("Prediction result:", pb_result)
# except grpc.RpcError as e:
#     print(f"gRPC error: {e.code()} - {e.details()}")

In [19]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc, predict_pb2
import tensorflow as tf
import numpy as np

# Определяем подключение к сервису
host = 'localhost:8502'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# Функция для преобразования numpy массива в protobuf
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, dtype=tf.float32)

# Создаем запрос
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

# Замените X на ваши данные
X = np.random.rand(1, 299, 299, 3).astype(np.float32)  # Пример случайных данных

pb_request.inputs['inputs'].CopyFrom(np_to_protobuf(X))

# Отправляем запрос
try:
    pb_result = stub.Predict(pb_request, timeout=20.0)
    print(pb_result)
except grpc.RpcError as e:
    print(f"Error: {e.code()}, {e.details()}")


Error: StatusCode.FAILED_PRECONDITION, Could not find variable block14_sepconv1_bn/moving_mean. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Resource localhost/block14_sepconv1_bn/moving_mean/N10tensorflow3VarE does not exist.
	 [[{{node functional_16_1/xception_1/block14_sepconv1_bn_1/Cast/ReadVariableOp}}]]
